## Character recognition 

This demo extends the kaggle [competition](https://www.kaggle.com/c/street-view-getting-started-with-julia/details/julia-tutorial). In this demo, we train using images of characters obtained from google view, and also classify test images which are of various sizes. Firstly k-Nearest Neighbor is used to identify the characters. Some of the take aways are how easy it is to build parallelizable and fast systems. We also try out various other methods and show comparisons. 

Download the following files from [here](https://www.kaggle.com/c/street-view-getting-started-with-julia/data), and place them in the /data directory :

* testResized/
* trainResized/
* sampleSubmission.csv
* trainLabels.csv

In [ ]:
using Images, Colors, DataFrames

In [ ]:
include("$(Pkg.dir())/MLDemos/src/knndemo.jl")

In [ ]:
# Some configurations
path = "$(Pkg.dir())/MLDemos/";
imageSize = 400
# Read the training labels
labelsInfoTrain = readtable("$(path)/data/trainLabels.csv")
# Read the test labels
labelsInfoTest = readtable("$(path)/data/testLabels.csv")


In [4]:
#read the images in from the training data.
xTrain = read_data("train", labelsInfoTrain, imageSize, path)
#read the test images
xTest = read_data("test", labelsInfoTest, imageSize, path)

"/Users/abhijithc/.julia/v0.4/MLDemos/demos"

In [29]:
#Map the training characters to ASCII values
yTrain = map(x -> x[1], labelsInfoTrain[:Class])
yTrain = int(yTrain)

read_data (generic function with 1 method)

In [ ]:
xTrain = xTrain'
xTest = xTest'

In [ ]:
# Add some processors
addprocs()

In [63]:
# Assign the labels to the training images and find the ratio
# of correctly classified labels to the total number of labels

k = 1
sumValues = @parallel (+) for i in 1:size(xTrain, 2)
 assign_label(xTrain, yTrain, k, i) == yTrain[i, 1]
end
loofCvAccuracy = sumValues / size(xTrain, 2)

2797

In [65]:
# Running the kNN on test data set
tic()
k = 3 # The CV accuracy shows this value to be the best.                             
yPredictions = @parallel (vcat) for i in 1:size(xTest, 2)
 nRows = size(xTrain, 1)
 imageI = Array(Float32, nRows)
 for index in 1:nRows
  imageI[index] = xTest[index, i]
 end
 assign_label(xTrain, yTrain, k, imageI)
end
toc()

0

In [ ]:
#Convert integer predictions to character                                            
labelsInfoTest[:Class] = char(yPredictions)

In [ ]:
#Save predictions                                                                    
writetable("$(path)/data/juliaKNNSubmission.csv", labelsInfoTest, separator=',', header=true)

In [ ]:
@show predictions = readtable("$(path)/data/juliaKNNSubmission.csv")